# Training ML models to predict CFD 


In [5]:
import os, glob, re, time
import copy as cp
import numpy as np
import statistics, scipy
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi']= 200
matplotlib.rcParams.update({'font.size': 5})

## <span style='background :yellow' > Import Shape Modelling PCA Data </span> 

In [6]:
ssm_pca_path = "/home/endrit/Documents/Modelling/COA/pipeline_1650/pca_u_new_shapes.csv"

In [7]:
ssm_file = open(ssm_pca_path, "r")
ssm_df = pd.read_csv(ssm_file, header=None, index_col=False)
ssm_pca_u = ssm_df.to_numpy()
print(np.shape(ssm_pca_u))

(1650, 66)


**keep only first 20 modes (99% cumvar)**

In [8]:
ssm_pca_u = ssm_pca_u[:, :20]
print(np.shape(ssm_pca_u))

(1650, 20)


**delete self-intersecting subjects**

In [9]:
ids_to_delete = [905, 1232, 1418, 1525]

ssm_pca_u = np.delete(ssm_pca_u, ids_to_delete, 0)
print(np.shape(ssm_pca_u))

(1646, 20)


## <span style='background :yellow' > Import CFD Data </span> 

In [10]:
###################################################################
# sorting function (human sorting for Shooting_0_, Shooting_1 etc.)
###################################################################
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    l.sort(key=alphanum_key)
    return l

In [11]:
cfd_path = "/home/endrit/Documents/Modelling/COA/pipeline_1650/Results_CFD/Point_Cloud_Interpolated/"

**import all filenames**

In [12]:
fnames = sort_nicely(os.listdir(cfd_path))

**import all csv flow fields (keep columns 1: pressure, 3: velocity)**

In [13]:
# build array of csv files
cfd_files = sort_nicely(os.listdir(cfd_path))
cfd_data = []
cfd_empty = [] # only x,y,z

for fn in cfd_files:
    file = open(cfd_path + fn, "r")
    df = pd.read_csv(file, index_col=False)
    df.columns = df.columns.str.replace(' ', '')
    # build list of cfd or empty dataframes
    df_cfd = df.drop(df.columns[[0, 2, 4, 5, 6, 7]], axis=1)
    df_empty = df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1)
    cfd_data.append(df_cfd.values)
    cfd_empty.append(df_empty.values)

In [14]:
# cfd_data[subject][node][pressure/velocity]
cfd_data = np.array(cfd_data)

# p_data[subject][node_pressure]
p_data = np.squeeze(cfd_data[:, :, :1])
v_data = np.squeeze(cfd_data[:, :, 1:])

In [15]:
# 594 subjects, 27420 pressure nodes (features)
np.shape(p_data)

(1646, 27420)

### <span style='background :yellow' > SPLIT DATASET </span> 

In [16]:
train_n = 1546

In [17]:
ssm_pca_u_train, ssm_pca_u_test = ssm_pca_u[:train_n], ssm_pca_u[train_n:]

p_data_train, p_data_test = p_data[:train_n], p_data[train_n:]
v_data_train, v_data_test = v_data[:train_n], v_data[train_n:]

# <span style='background :pink' > PCA ON CFD *TRAINING* DATA </span> 

##### matrix = 1546 rows (training subjects), 27272 columns (variables/features)

NOTE: sklearn.decomposition.PCA does **NOT** scale the features as part of function. It only centers them by default. Scaling may or may not be necessary (centering + scaling = standardisation).

NOTE: Scaling the datasets made it much more difficult to capture the variance with fewer modes. This may or may not have an effect on the result. All features have the same units, so maybe the outcome is not affected.

### Pressure

In [18]:
# center each feature => mean = 0
scaler_p = StandardScaler(with_std=False)
# fit scaler on TRAINING set
p_data_train_scaled = scaler_p.fit_transform(p_data_train)
p_data_test_scaled = scaler_p.transform(p_data_test)

In [19]:
# compute PCA
U_p, S_p, Vt_p = np.linalg.svd(p_data_train_scaled, full_matrices=False)

In [20]:
'''
Get cumulative variance (start from mode 0)
Keep adding modes until 99% variance captured
'''
cumvar_p = np.cumsum(S_p**2) / sum(S_p**2)
cumvar_p[:15]

array([0.49060472, 0.78824798, 0.89725592, 0.93479715, 0.95578857,
       0.96759894, 0.97542911, 0.98153384, 0.98599139, 0.98807608,
       0.98963679, 0.99096671, 0.99203806, 0.99275675, 0.99337529])

In [21]:
n_modes_p = 15

In [22]:
'''
U matrix is [subjects, scores(per mode)]
first 10 columns are the 10 modes capturing most variability
'''
p_data_pca_train = (U_p[:, :n_modes_p])
p_data_pca_train.shape

(1546, 15)

### Velocity

In [23]:
# standardise each feature => mean = 0
scaler_v = StandardScaler(with_std=False)
v_data_train_scaled = scaler_v.fit_transform(v_data_train)
v_data_test_scaled = scaler_v.transform(v_data_test)

In [24]:
# compute PCA
U_v, S_v, Vt_v = np.linalg.svd(v_data_train_scaled, full_matrices=False)

In [25]:
'''
Capturing variance much more difficult for velocity
30 modes = 93% of var
'''
cumvar_v = np.cumsum(S_v**2) / sum(S_v**2)
cumvar_v[:30]

array([0.5940733 , 0.72006929, 0.77743264, 0.82587701, 0.84773948,
       0.86094625, 0.87274331, 0.88295761, 0.8905191 , 0.89675573,
       0.90107312, 0.90476839, 0.90775381, 0.91027186, 0.91244345,
       0.91451058, 0.91639456, 0.9180798 , 0.91957441, 0.92098284,
       0.92224203, 0.92343444, 0.92454152, 0.92557572, 0.9265165 ,
       0.92741592, 0.92829345, 0.92907845, 0.92984204, 0.93057657])

In [26]:
n_modes_v = 30

In [27]:
# transform velocity data -> pca modes
v_data_pca_train = (U_v[:, :n_modes_v])
v_data_pca_train.shape

(1546, 30)

### Finalise train and test datasets

In [28]:
'''
scaler_ssm = StandardScaler()
ssm_train_scaled = scaler_ssm.fit_transform(ssm_pca_u_train)
ssm_test_scaled = scaler_ssm.transform(ssm_pca_u_test)
'''

'\nscaler_ssm = StandardScaler()\nssm_train_scaled = scaler_ssm.fit_transform(ssm_pca_u_train)\nssm_test_scaled = scaler_ssm.transform(ssm_pca_u_test)\n'

In [29]:
#X_train, X_test = ssm_train_scaled, ssm_test_scaled
X_train, X_test = ssm_pca_u_train, ssm_pca_u_test

In [30]:
P_train, P_test = p_data_train_scaled, p_data_test_scaled
V_train, V_test = v_data_train_scaled, v_data_test_scaled

In [31]:
print("X_train: " + str(X_train.shape) + " X_test: " + str(X_test.shape))
print("")
print("P_train: " + str(P_train.shape) + " X_test: " + str(P_test.shape))
print("V_train: " + str(V_train.shape) + " X_test: " + str(V_test.shape))

X_train: (1546, 20) X_test: (100, 20)

P_train: (1546, 27420) X_test: (100, 27420)
V_train: (1546, 27420) X_test: (100, 27420)


# <span style='background :pink' > DEEP LEARNING </span> 

In [32]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [33]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version: " + str(tf.__version__))
print("keras version: " + str(keras.__version__))

Num GPUs Available:  1
tensorflow version: 2.4.1
keras version: 2.4.0


**define lambda layer for inverse PCA as a DNN layer**

In [34]:
# get S and V matrices in tensor type
S_p_tensor = tf.convert_to_tensor(S_p)
Vt_p_tensor = tf.convert_to_tensor(Vt_p)
S_v_tensor = tf.convert_to_tensor(S_v)
Vt_v_tensor = tf.convert_to_tensor(Vt_v)

In [35]:
def inv_pressure_pca(x):
    x = tf.cast(x, tf.float64)
    n_modes = x.get_shape().as_list()[1]
    x_inv = tf.matmul(tf.matmul(x, tf.linalg.diag(S_p_tensor[:n_modes])), Vt_p_tensor[:n_modes, :])
    return x_inv

def inv_velocity_pca(x):
    x = tf.cast(x, tf.float64)
    n_modes = x.get_shape().as_list()[1]
    x_inv = tf.matmul(tf.matmul(x, tf.linalg.diag(S_v_tensor[:n_modes])), Vt_v_tensor[:n_modes, :])
    return x_inv

In [32]:
# testing loss function
'''
test_loss_pred = tf.reshape(tf.convert_to_tensor(p_data_pca_train[0]), [1, 15])
test_loss_pred = tf.math.multiply(test_loss_pred, 1.1)
test_loss_true = tf.reshape(tf.convert_to_tensor(p_data_pca_train[0]), [1, 15])
print(inv_pressure_pca(test_loss_pred))
'''

'\ntest_loss_pred = tf.reshape(tf.convert_to_tensor(p_data_pca_train[0]), [1, 15])\ntest_loss_pred = tf.math.multiply(test_loss_pred, 1.1)\ntest_loss_true = tf.reshape(tf.convert_to_tensor(p_data_pca_train[0]), [1, 15])\nprint(inv_pressure_pca(test_loss_pred))\n'

# <span style='background :lightgreen' > HYPERPARAMETER OPTIMISATION </span> 

In [36]:
import optuna
import plotly
from sklearn.model_selection import KFold

### <span style='background :lightgreen' > PRESSURE </span> 

In [37]:
# define model
def build_model_p(trial):
    INPUT = 20
    OUTPUT = n_modes_p
    # hyperparameters
    NUM_LAYERS = trial.suggest_int("n_layers", 2, 5, step=1)
    NUM_UNITS = trial.suggest_int("n_units", 30, 100, step=10)
    LEARN_RATE = trial.suggest_loguniform("learn_rate", 1e-3, 1e-2)
    # build model
    model = keras.models.Sequential()
    # input layer
    model.add(keras.layers.Dense(INPUT, activation="relu"))
    # hidden layers
    for layer in range(NUM_LAYERS):
        model.add(keras.layers.Dense(NUM_UNITS, activation="relu"))                                     
    # output layer
    model.add(keras.layers.Dense(OUTPUT, activation="linear"))
    # inverse pca
    model.add(keras.layers.Lambda(inv_pressure_pca))
    # optimiser and loss
    # steps_per_epoch = training set / batch size = 1236 / 128 = 10
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARN_RATE, 
                                                              decay_steps=10, decay_rate=0.95)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss="mae", optimizer=optimizer, run_eagerly=True)
    
    return model

In [38]:
def objective_p(trial):
    
    NUM_FOLDS = 5
    EPOCHS = 50
    BATCH_SIZE = 128
    kfold = KFold(n_splits=NUM_FOLDS)
    
    cv_scores=[]
    for train_i, test_i in kfold.split(X_train):
        # pressure data
        x_train, x_valid = X_train[train_i], X_train[test_i]
        y_train, y_valid = P_train[train_i], P_train[test_i]
        
        model = build_model_p(trial)
        
        model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        validation_data=(x_valid, y_valid),
        epochs=EPOCHS,
        callbacks=[optuna.integration.TFKerasPruningCallback(trial, "val_loss"),
                   tf.keras.callbacks.EarlyStopping(patience=10)],
        verbose=1)
        
        score = model.evaluate(x_valid, y_valid, verbose=0)
        cv_scores.append(score)
    
    return np.mean(cv_scores)

In [39]:
study_p = optuna.create_study(direction="minimize", 
                              sampler=optuna.samplers.TPESampler(), 
                              pruner=optuna.pruners.HyperbandPruner())

study_p.optimize(objective_p, n_trials=100)

[I 2022-02-10 18:34:24,649] A new study created in memory with name: no-name-5119899c-6ac5-4cd1-84b8-7ac2794a52f4


Epoch 1/50
10/10 [==============================] - 9s 140ms/step - loss: 4647.6781 - val_loss: 2742.2056
Epoch 2/50
10/10 [==============================] - 0s 42ms/step - loss: 2767.3614 - val_loss: 2679.2444
Epoch 3/50
10/10 [==============================] - 0s 44ms/step - loss: 2587.5297 - val_loss: 2233.4753
Epoch 4/50
10/10 [==============================] - 0s 43ms/step - loss: 2171.7559 - val_loss: 1888.9335
Epoch 5/50
10/10 [==============================] - 0s 43ms/step - loss: 1799.2664 - val_loss: 1554.2491
Epoch 6/50
10/10 [==============================] - 0s 44ms/step - loss: 1524.6935 - val_loss: 1383.8046
Epoch 7/50
10/10 [==============================] - 0s 43ms/step - loss: 1394.6727 - val_loss: 1291.1438
Epoch 8/50
10/10 [==============================] - 0s 42ms/step - loss: 1289.8411 - val_loss: 1212.7029
Epoch 9/50
10/10 [==============================] - 0s 42ms/step - loss: 1199.6306 - val_loss: 1165.4183
Epoch 10/50
10/10 [==============================] - 0

/home/endrit/anaconda3/envs/keras-env/lib/python3.9/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


10/10 [==============================] - 0s 45ms/step - loss: 2205.7143 - val_loss: 2000.0840
Epoch 3/50
 5/10 [==============>...............] - ETA: 0s - loss: 1833.3345

/home/endrit/anaconda3/envs/keras-env/lib/python3.9/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


...

[I 2022-02-10 19:07:49,058] Trial 97 pruned. Trial was pruned at epoch 1.


Epoch 1/50
10/10 [==============================] - 1s 60ms/step - loss: 5113.8305 - val_loss: 2811.0371
Epoch 2/50
10/10 [==============================] - 0s 41ms/step - loss: 2685.1681 - val_loss: 2434.5977


[I 2022-02-10 19:07:50,655] Trial 98 pruned. Trial was pruned at epoch 1.


Epoch 1/50
10/10 [==============================] - 1s 63ms/step - loss: 3351.4153 - val_loss: 2643.2500
Epoch 2/50
10/10 [==============================] - 0s 43ms/step - loss: 2478.5403 - val_loss: 2165.1021
Epoch 3/50
10/10 [==============================] - 0s 43ms/step - loss: 2066.0037 - val_loss: 1900.1129
Epoch 4/50
10/10 [==============================] - 0s 45ms/step - loss: 1802.3757 - val_loss: 1670.0813
Epoch 5/50
10/10 [==============================] - 0s 44ms/step - loss: 1624.5258 - val_loss: 1593.9170
Epoch 6/50
10/10 [==============================] - 0s 44ms/step - loss: 1476.9707 - val_loss: 1527.8041
Epoch 7/50
10/10 [==============================] - 0s 44ms/step - loss: 1347.6921 - val_loss: 1374.5669
Epoch 8/50
10/10 [==============================] - 0s 43ms/step - loss: 1267.1924 - val_loss: 1366.7502
Epoch 9/50
10/10 [==============================] - 0s 43ms/step - loss: 1203.7048 - val_loss: 1290.7917
Epoch 10/50
10/10 [==============================] - 0s

[I 2022-02-10 19:08:03,504] Trial 99 pruned. Trial was pruned at epoch 27.


In [40]:
pruned_trials_p = study_p.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
complete_trials_p = study_p.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])

In [ ]:
optuna.visualization.plot_optimization_history(study_p)

In [ ]:
optuna.visualization.plot_intermediate_values(study_p)

In [ ]:
optuna.visualization.plot_slice(study_p)

In [ ]:
optuna.visualization.plot_contour(study_p)

In [ ]:
optuna.visualization.plot_param_importances(study_p)

In [55]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study_p.trials))
print("  Number of pruned trials: ", len(pruned_trials_p))
print("  Number of complete trials: ", len(complete_trials_p))

print("Best trial:")
trial = study_p.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  84
  Number of complete trials:  16
Best trial:
  Value:  801.37177734375
  Params: 
    n_layers: 4
    n_units: 90
    learn_rate: 0.0062983534252622


### <span style='background :lightgreen' > VELOCITY </span> 

In [39]:
# define model
def build_model_v(trial):
    INPUT = 20
    OUTPUT = n_modes_v
    # hyperparameters
    NUM_LAYERS = trial.suggest_int("n_layers", 2, 5, step=1)
    NUM_UNITS = trial.suggest_int("n_units", 30, 100, step=10)
    LEARN_RATE = trial.suggest_loguniform("learn_rate", 1e-3, 1e-2)
    
    # build model
    model = keras.models.Sequential()
    # input layer
    model.add(keras.layers.Dense(INPUT, activation="relu"))
    # hidden layers
    for layer in range(NUM_LAYERS):
        model.add(keras.layers.Dense(NUM_UNITS, activation="relu"))                                     
    # output layer
    model.add(keras.layers.Dense(OUTPUT, activation="linear"))
    # inverse pca
    model.add(keras.layers.Lambda(inv_velocity_pca))
    # optimiser and loss
    # steps_per_epoch = training set / batch size
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARN_RATE, 
                                                              decay_steps=10, decay_rate=0.95)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss="mae", optimizer=optimizer, run_eagerly=True)
    
    return model

In [40]:
def objective_v(trial):
    
    NUM_FOLDS = 5
    EPOCHS = 50
    BATCH_SIZE = 128
    kfold = KFold(n_splits=NUM_FOLDS)
    
    cv_scores=[]
    for train_i, test_i in kfold.split(X_train):
        # pressure data
        x_train, x_valid = X_train[train_i], X_train[test_i]
        y_train, y_valid = V_train[train_i], V_train[test_i]
        
        model = build_model_v(trial)
        
        model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        validation_data=(x_valid, y_valid),
        epochs=EPOCHS,
        callbacks=[optuna.integration.TFKerasPruningCallback(trial, "val_loss"),
                   tf.keras.callbacks.EarlyStopping(patience=10)],
        verbose=1)
        
        score = model.evaluate(x_valid, y_valid, verbose=0)
        cv_scores.append(score)
    
    return np.mean(cv_scores)

In [41]:
study_v = optuna.create_study(direction="minimize", 
                              sampler=optuna.samplers.TPESampler(), 
                              pruner=optuna.pruners.HyperbandPruner())

study_v.optimize(objective_v, n_trials=100)

[I 2022-02-11 13:18:53,121] A new study created in memory with name: no-name-3e58fbfc-3145-4419-bb8f-6b93cfbc86e6


Epoch 1/50
10/10 [==============================] - 3s 125ms/step - loss: 0.7233 - val_loss: 0.6124
Epoch 2/50
10/10 [==============================] - 0s 41ms/step - loss: 0.6194 - val_loss: 0.6124
Epoch 3/50
10/10 [==============================] - 0s 41ms/step - loss: 0.6111 - val_loss: 0.6123
Epoch 4/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6108 - val_loss: 0.6123
Epoch 5/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6131 - val_loss: 0.6123
Epoch 6/50
10/10 [==============================] - 0s 41ms/step - loss: 0.6160 - val_loss: 0.6123
Epoch 7/50
10/10 [==============================] - 0s 41ms/step - loss: 0.6095 - val_loss: 0.6123
Epoch 8/50
10/10 [==============================] - 0s 42ms/step - loss: 0.6298 - val_loss: 0.6123
Epoch 9/50
10/10 [==============================] - 0s 42ms/step - loss: 0.6235 - val_loss: 0.6123
Epoch 10/50
10/10 [==============================] - 0s 42ms/step - loss: 0.6239 - val_loss: 0.6123
Epoch 11

/home/endrit/anaconda3/envs/keras-env/lib/python3.9/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


10/10 [==============================] - 0s 42ms/step - loss: 0.5940 - val_loss: 0.6100
Epoch 3/50
 5/10 [==============>...............] - ETA: 0s - loss: 0.5799

/home/endrit/anaconda3/envs/keras-env/lib/python3.9/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


...

[I 2022-02-11 13:42:40,140] Trial 97 pruned. Trial was pruned at epoch 1.


Epoch 1/50
10/10 [==============================] - 1s 62ms/step - loss: 0.6353 - val_loss: 0.6122
Epoch 2/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6237 - val_loss: 0.6101
Epoch 3/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6069 - val_loss: 0.6068
Epoch 4/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6180 - val_loss: 0.6031
Epoch 5/50
10/10 [==============================] - 0s 44ms/step - loss: 0.6108 - val_loss: 0.6009
Epoch 6/50
10/10 [==============================] - 0s 44ms/step - loss: 0.6047 - val_loss: 0.5997
Epoch 7/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6067 - val_loss: 0.5976
Epoch 8/50
10/10 [==============================] - 0s 43ms/step - loss: 0.6031 - val_loss: 0.5966
Epoch 9/50
10/10 [==============================] - 0s 44ms/step - loss: 0.6036 - val_loss: 0.5933
Epoch 10/50
10/10 [==============================] - 0s 43ms/step - loss: 0.5907 - val_loss: 0.5913


[I 2022-02-11 13:42:45,178] Trial 98 pruned. Trial was pruned at epoch 9.


Epoch 1/50
10/10 [==============================] - 1s 59ms/step - loss: 0.6735 - val_loss: 0.6125
Epoch 2/50
10/10 [==============================] - 0s 41ms/step - loss: 0.6194 - val_loss: 0.6125


[I 2022-02-11 13:42:46,894] Trial 99 pruned. Trial was pruned at epoch 1.


In [42]:
pruned_trials_v = study_v.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
complete_trials_v = study_v.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])

In [ ]:
optuna.visualization.plot_optimization_history(study_v)

In [ ]:
optuna.visualization.plot_intermediate_values(study_v)

In [ ]:
optuna.visualization.plot_slice(study_v)

In [ ]:
optuna.visualization.plot_contour(study_v)

In [ ]:
optuna.visualization.plot_param_importances(study_v)

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study_v.trials))
print("  Number of pruned trials: ", len(pruned_trials_v))
print("  Number of complete trials: ", len(complete_trials_v))

print("Best trial:")
trial = study_v.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# <span style='background :red' > TRAINING TUNED MODELS </span> 

In [49]:
run_attempt = 4

**TensorBoard Path**

In [50]:
root_logdir = os.getcwd() + "/Results_ML/Tensorflow/my_logs/"

# make a path using date/time
def get_run_logdir(metric):
    run_id = "run " + str(run_attempt) + "__" + time.strftime("run_%Y_%m_%d__%H-%M") + "__" + metric
    return os.path.join(root_logdir, run_id)

In [51]:
tensorboard_p_logdir = get_run_logdir("Pressure")
tensorboard_v_logdir = get_run_logdir("Velocity")

**DNN Model Path**

In [54]:
model_dir = os.getcwd() + "/Results_ML/Run_" + str(run_attempt) + "/model/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

**Define and Train Models**

In [70]:
def build_tuned_pressure_model():
    INPUT = 20
    OUTPUT = n_modes_p
    # hyperparameters from tuning
    NUM_LAYERS = study_p.best_trial.params["n_layers"]
    NUM_UNITS = study_p.best_trial.params["n_units"]
    LEARN_RATE = study_p.best_trial.params["learn_rate"]
    # model
    model = keras.models.Sequential()
    # input layer
    model.add(keras.layers.Dense(INPUT, activation="relu"))
    # hidden layers
    for layer in range(NUM_LAYERS):
        model.add(keras.layers.Dense(NUM_UNITS, activation="relu"))                                     
    # output layer - cfd pca
    model.add(keras.layers.Dense(OUTPUT, activation="linear"))
    # pca conversion
    model.add(keras.layers.Lambda(inv_pressure_pca))
    # optimiser
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARN_RATE, 
                                                              decay_steps=11, decay_rate=0.95)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    # compile
    model.compile(loss="mae", optimizer=optimizer, run_eagerly=True)
    
    return model

In [71]:
# train pressure NN
dnn_p = build_tuned_pressure_model()

dnn_p.fit(X_train, 
          P_train, 
          epochs=100, 
          batch_size=128,
          validation_split=0.1, 
          callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                   patience=10), 
                     keras.callbacks.ModelCheckpoint(model_dir+"/dnn_p.h5", 
                                                     monitor="val_loss", 
                                                     save_best_only=True),
                     keras.callbacks.TensorBoard(log_dir=tensorboard_p_logdir, 
                                                 histogram_freq=1)])

Epoch 1/100
11/11 [==============================] - 1s 98ms/step - loss: 4172.7161 - val_loss: 2737.7883
Epoch 2/100
11/11 [==============================] - 0s 41ms/step - loss: 2778.4882 - val_loss: 2342.5144
Epoch 3/100
11/11 [==============================] - 0s 41ms/step - loss: 2370.1953 - val_loss: 1953.6460
Epoch 4/100
11/11 [==============================] - 0s 39ms/step - loss: 1852.3233 - val_loss: 1735.4066
Epoch 5/100
11/11 [==============================] - 0s 41ms/step - loss: 1712.8866 - val_loss: 1567.0657
Epoch 6/100
11/11 [==============================] - 0s 43ms/step - loss: 1409.8146 - val_loss: 1439.4602
Epoch 7/100
11/11 [==============================] - 0s 39ms/step - loss: 1337.7147 - val_loss: 1360.1455
Epoch 8/100
11/11 [==============================] - 0s 40ms/step - loss: 1230.8939 - val_loss: 1317.0887
Epoch 9/100
11/11 [==============================] - 0s 40ms/step - loss: 1174.0021 - val_loss: 1236.3115
Epoch 10/100
11/11 [==========================

Epoch 79/100
11/11 [==============================] - 0s 43ms/step - loss: 590.3322 - val_loss: 789.6227
Epoch 80/100
11/11 [==============================] - 0s 42ms/step - loss: 602.8335 - val_loss: 784.6053
Epoch 81/100
11/11 [==============================] - 1s 46ms/step - loss: 597.1039 - val_loss: 785.5032
Epoch 82/100
11/11 [==============================] - 0s 44ms/step - loss: 588.9407 - val_loss: 784.5375
Epoch 83/100
11/11 [==============================] - 0s 43ms/step - loss: 575.1724 - val_loss: 787.1806
Epoch 84/100
11/11 [==============================] - 0s 46ms/step - loss: 586.1310 - val_loss: 787.9330
Epoch 85/100
11/11 [==============================] - 0s 41ms/step - loss: 585.0384 - val_loss: 785.8589
Epoch 86/100
11/11 [==============================] - 0s 41ms/step - loss: 578.4294 - val_loss: 785.1852
Epoch 87/100
11/11 [==============================] - 0s 40ms/step - loss: 591.6152 - val_loss: 785.8019
Epoch 88/100
11/11 [==============================] - 0

In [52]:
def build_tuned_velocity_model():
    INPUT = 20
    OUTPUT = n_modes_v
    # hyperparameters from tuning
    NUM_LAYERS = study_v.best_trial.params["n_layers"]
    NUM_UNITS = study_v.best_trial.params["n_units"]
    LEARN_RATE = study_v.best_trial.params["learn_rate"]
    # model
    model = keras.models.Sequential()
    # input layer
    model.add(keras.layers.Dense(INPUT, activation="relu"))
    # hidden layers
    for layer in range(NUM_LAYERS):
        model.add(keras.layers.Dense(NUM_UNITS, activation="relu"))                                     
    # output layer - cfd pca
    model.add(keras.layers.Dense(OUTPUT, activation="linear"))
    # pca conversion
    model.add(keras.layers.Lambda(inv_velocity_pca))
    # optimiser
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARN_RATE, 
                                                              decay_steps=11, decay_rate=0.95)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    # compile
    model.compile(loss="mae", optimizer=optimizer, run_eagerly=True)
    
    return model

In [55]:
# train velocity NN
dnn_v = build_tuned_velocity_model()

dnn_v.fit(X_train, 
          V_train, 
          batch_size = 128,
          epochs=100, 
          validation_split=0.1, 
          callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                   patience=100), 
                     keras.callbacks.ModelCheckpoint(model_dir+"/dnn_v.h5", 
                                                     monitor="val_loss", 
                                                     save_best_only=True),
                     keras.callbacks.TensorBoard(log_dir=tensorboard_v_logdir, 
                                                 histogram_freq=1)])

Epoch 1/100
11/11 [==============================] - 1s 75ms/step - loss: 0.6111 - val_loss: 0.6206
Epoch 2/100
11/11 [==============================] - 0s 41ms/step - loss: 0.6098 - val_loss: 0.6205
Epoch 3/100
11/11 [==============================] - 0s 40ms/step - loss: 0.6168 - val_loss: 0.6192
Epoch 4/100
11/11 [==============================] - 0s 40ms/step - loss: 0.6079 - val_loss: 0.6139
Epoch 5/100
11/11 [==============================] - 0s 40ms/step - loss: 0.6011 - val_loss: 0.6043
Epoch 6/100
11/11 [==============================] - 0s 40ms/step - loss: 0.5981 - val_loss: 0.6022
Epoch 7/100
11/11 [==============================] - 0s 41ms/step - loss: 0.5961 - val_loss: 0.5934
Epoch 8/100
11/11 [==============================] - 0s 41ms/step - loss: 0.5860 - val_loss: 0.5902
Epoch 9/100
11/11 [==============================] - 0s 41ms/step - loss: 0.5833 - val_loss: 0.5831
Epoch 10/100
11/11 [==============================] - 0s 42ms/step - loss: 0.5751 - val_loss: 0.5640

**Tensorboard Visualisation**

In [74]:
# remember, set smoothing to 0
%reload_ext tensorboard
%tensorboard --logdir=./Results_ML/Tensorflow/my_logs/ --port=6008

**Saving Pipeline**

In [75]:
'''
from sklearn.pipeline import Pipeline
import joblib

# save scalers in model dir
pipeline = Pipeline([
    # output
    ('scaler_p', scaler_p),
    ('scaler_v', scaler_v)
])
joblib.dump(pipeline, model_dir + 'pipeline.pkl')

# save U, S and Vt matrices in model dir
np.save(model_dir+'U_p', U_p, allow_pickle=True)
np.save(model_dir+'S_p', S_p, allow_pickle=True)
np.save(model_dir+'Vt_p', Vt_p, allow_pickle=True)
np.save(model_dir+'U_v', U_v, allow_pickle=True)
np.save(model_dir+'S_v', S_v, allow_pickle=True)
np.save(model_dir+'Vt_v', Vt_v, allow_pickle=True)
'''

# <span style='background :cyan' > DNN PREDICTION </span> 

In [84]:
##############################################################################
# IF LOADING MODELS
##############################################################################

from sklearn.pipeline import Pipeline
import joblib

run_attempt = 3

model_dir = os.getcwd() + "/Results_ML/Run_" + str(run_attempt) + "/model/"

# load S and Vt matrices (needed for estimators)
S_p_tensor = tf.convert_to_tensor(np.load(model_dir + 'S_p.npy'))
S_v_tensor = tf.convert_to_tensor(np.load(model_dir + 'S_v.npy'))
Vt_p_tensor = tf.convert_to_tensor(np.load(model_dir + 'Vt_p.npy'))
Vt_v_tensor = tf.convert_to_tensor(np.load(model_dir + 'Vt_v.npy'))

# load pipeline
pipeline_loaded = joblib.load(model_dir + 'pipeline.pkl')
scaler_p = pipeline_loaded[0]
scaler_v = pipeline_loaded[1]

# load estimators
dnn_p = keras.models.load_model(model_dir + "dnn_p.h5")
dnn_v = keras.models.load_model(model_dir + "dnn_v.h5")

In [85]:
dnn_p.summary()

Model: "sequential_324"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1762 (Dense)           (None, 20)                420       
_________________________________________________________________
dense_1763 (Dense)           (None, 90)                1890      
_________________________________________________________________
dense_1764 (Dense)           (None, 90)                8190      
_________________________________________________________________
dense_1765 (Dense)           (None, 90)                8190      
_________________________________________________________________
dense_1766 (Dense)           (None, 90)                8190      
_________________________________________________________________
dense_1767 (Dense)           (None, 15)                1365      
_________________________________________________________________
lambda_324 (Lambda)          (None, 27420)          

In [86]:
dnn_v.summary()

Model: "sequential_325"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1768 (Dense)           (None, 20)                420       
_________________________________________________________________
dense_1769 (Dense)           (None, 90)                1890      
_________________________________________________________________
dense_1770 (Dense)           (None, 90)                8190      
_________________________________________________________________
dense_1771 (Dense)           (None, 90)                8190      
_________________________________________________________________
dense_1772 (Dense)           (None, 30)                2730      
_________________________________________________________________
lambda_325 (Lambda)          (None, 27420)             0         
Total params: 21,420
Trainable params: 21,420
Non-trainable params: 0
________________________________________________

**Pressure**  

In [93]:
time_1 = time.time()

# predict standardised data
P_pred = dnn_p.predict(X_test)

# reverse standardisation
P_pred_inv_scaling = scaler_p.inverse_transform(P_pred)

time_2 = time.time()
print('{} seconds'.format(time_2 - time_1))

0.5579886436462402 seconds


In [94]:
np.shape(P_pred)

(100, 27420)

**Velocity**

In [153]:
time_1 = time.time()

# predict standardised data
V_pred = dnn_v.predict(X_test)

# inverse standardisation 
V_pred_inv_scaling = scaler_v.inverse_transform(V_pred)
# make all negatives = 0
V_pred_inv_scaling_no_negatives = np.clip(V_pred_inv_scaling, a_min=0, a_max=None)

time_2 = time.time()
print('{} seconds'.format(time_2 - time_1))

0.5568013191223145 seconds


In [154]:
np.shape(V_pred_inv_scaling)

(100, 27420)

**Build new files**

In [155]:
pred_empty_csv = cfd_empty[train_n:]

In [156]:
# add predictions to csv
pred_csv = np.dstack((pred_empty_csv, P_pred_inv_scaling))
pred_csv = np.dstack((pred_csv, V_pred_inv_scaling_no_negatives))

# calculate P and V errors and add them to csv
errors_p, errors_v = [], []
for i in range(len(pred_csv)):
    err_p = abs(np.subtract(abs(pred_csv[i, :, 3]), abs((p_data[i+len(X_train), :]))))
    err_v = abs(np.subtract(abs(pred_csv[i, :, 4]), abs((v_data[i+len(X_train), :]))))
    errors_p.append(err_p)
    errors_v.append(err_v)
    
pred_csv = np.dstack((pred_csv, errors_p))
pred_csv = np.dstack((pred_csv, errors_v))

In [157]:
np.shape(pred_csv)

(100, 27420, 7)

# <span style='background :cyan' > DNN EVALUATION </span> 

In [158]:
test_filenames = fnames[train_n:]
test_filenames = [s.replace("_ext.csv", "") for s in test_filenames]

In [159]:
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

pressure_pred_evaluation = pd.DataFrame(columns=['filename', 'mean_true (Pa)', 'error_MAE (Pa)', 'MAE/mean (%)'])
for i in range(p_data_test.shape[0]):
    MAE = mean_absolute_error(p_data_test[i], P_pred_inv_scaling[i])
    mean = np.mean(p_data_test[i])
    pressure_pred_evaluation.loc[i] = [test_filenames[i], mean, MAE, MAE*100/mean]
    
print(pressure_pred_evaluation)
print("mean of MAE is = " + "{:.2f}".format(np.mean(pressure_pred_evaluation['error_MAE (Pa)'])) + " Pa")
print("mean of MAE/mean is = " + "{:.2f}".format(np.mean(pressure_pred_evaluation['MAE/mean (%)'])) + " %")


velocity_pred_evaluation = pd.DataFrame(columns=['filename', 'mean_true (m/s)', 'error_MAE (m/s)', 'MAE/mean (%)'])
for i in range(v_data_test.shape[0]):
    MAE = mean_absolute_error(v_data_test[i], V_pred_inv_scaling_no_negatives[i])
    mean = np.mean(v_data_test[i])
    velocity_pred_evaluation.loc[i] = [test_filenames[i], mean, MAE, MAE*100/mean]

print("")
print(velocity_pred_evaluation)
print("mean of MAE is = " + "{:.2f}".format(np.mean(velocity_pred_evaluation['error_MAE (m/s)'])) + " m/s")
print("mean of MAE/mean is = " + "{:.2f}".format(np.mean(velocity_pred_evaluation['MAE/mean (%)'])) + " %")

         filename  mean_true (Pa)  error_MAE (Pa)  MAE/mean (%)
0   Shooting_1550     3279.641875      618.581724     18.861258
1   Shooting_1551     2746.937909      387.492941     14.106360
2   Shooting_1552     6161.775604     1328.764566     21.564637
3   Shooting_1553     2154.945881      349.238280     16.206360
4   Shooting_1554     3982.940409      496.883730     12.475299
..            ...             ...             ...           ...
95  Shooting_1645      901.723730      401.631489     44.540415
96  Shooting_1646     4844.031224      833.748411     17.211871
97  Shooting_1647     5185.642382      338.446039      6.526598
98  Shooting_1648     2534.356920      299.872663     11.832298
99  Shooting_1649     2037.980510      637.588661     31.285317

[100 rows x 4 columns]
mean of MAE is = 639.07 Pa
mean of MAE/mean is = 2.59 %

         filename  mean_true (m/s)  error_MAE (m/s)  MAE/mean (%)
0   Shooting_1550         1.677628         0.171075     10.197462
1   Shooting_1551  

### Saving Predictions

In [160]:
data_dir = model_dir.replace("model/", "predictions/")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(data_dir)

/home/endrit/Documents/Modelling/COA/pipeline_1650/Results_ML/Run_3/predictions/


**Save csv file of metrics**

In [161]:
pressure_pred_evaluation.to_csv(data_dir+"evaluation_pressure.csv", sep='\t')
velocity_pred_evaluation.to_csv(data_dir+"evaluation_velocity.csv", sep='\t')

**Save predicted point clouds as csv**

In [162]:
for i in range(len(pred_csv)):
    pred_full_csv = pd.DataFrame(pred_csv[i])
    pred_full_csv.rename(columns={0: 'x', 1: 'y', 2: 'z', 3: 'Pressure_pred_(Pa)', 4: 'Velocity_pred_(m/s)', 
                            5: 'Pressure_error_(Pa)', 6: 'Velocity_error_(m/s)'}, inplace=True)
    pred_full_csv.to_csv(data_dir + "pred_" + str(fnames[i+train_n]) + ".csv", index=False)